In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tkinter import *
from tkinter import messagebox as tkMessageBox
from tkinter import filedialog
import threading
import time
import re

from collections import OrderedDict
#Above all the libraries, word after "from" is library name which is to installed and word after"import" is function in the library
def create_dict(data,years):
    # This function create the dictionary function in python like say
    # we have data: ["2020","2020","2019] and say data is ["Jun2020","May2020","Apr2019"]
    #So this function creates : {"2020":["Jun2020","May2020"],"2019":["Apr2019"]
    dictio=OrderedDict()
    for t in range(len(years)):
      if years[t] in dictio:
        dictio[years[t]].append(data[t])
      else:
           dictio[years[t]]=[data[t]]
    return dictio
import pygsheets
import pandas as pd
def file_generate(link,mont,index,aum_sheet,city_sheet):
    # This function is for putting data into the google sheets
    temp=pd.ExcelFile(link)
    file1=pd.read_excel(temp,sheet_name=0)
    file1.fillna(" ",inplace=True)
    if index==0:
        # this if condition is when value of index is 0, i.e first sheet in google spreadsheet
        # This is for aum distribution according to distribution method
        aum_sheet.sheet1.title=mont
        wks = aum_sheet.worksheet(property='index', value=index)
        wks.set_dataframe(file1,start='A1')
    else:
        #for adding other sheets in the spreadhsheet
        #This is for aum distribution according to distribution method
        aum_sheet.add_worksheet(mont)
        wks = aum_sheet.worksheet(property='index', value=index)
        wks.set_dataframe(file1, start='A1')
    file1 = pd.read_excel(temp, sheet_name=1)
    file1.fillna(" ",inplace=True)
    if index == 0:
        # This is for State-wise sheet in a different spreadsheet than distribution sheet above
        city_sheet.sheet1.title = mont
        wks = city_sheet.worksheet(property='index', value=index)
        wks.set_dataframe(file1, start='A1')
    else:
        city_sheet.add_worksheet(mont)
        wks = city_sheet.worksheet(property='index', value=index)
        wks.set_dataframe(file1, start='A1')

class GUI_python:
    # This class is for creating the GUI
    def gui_guide(self):
        self.frame = Tk()
        self.frame.resizable(0, 0)
        self.frame.title("HDFC AMC AUM/Statewise Data")
        self.frame.geometry('900x200')
        # Above few lines of codes create a window
        self.emailFrame = Frame(self.frame, width=200, height=600, highlightthickness=2,
                                highlightbackground="#111")
        # This creates a frame which creates a box around label which has enter email id text
        self.emailFrame.grid(row=1,column=0, padx=10, pady=2, sticky=N + S)
        self.servicFrame = Frame(self.frame, width=200, height=300, highlightthickness=2,
                                  highlightbackground="#111")
        self.servicFrame.grid(row=2, column=0, padx=10, pady=2, sticky=N + S)
        self.driverFrame = Frame(self.frame, width=200, height=300, highlightthickness=2,
                                 highlightbackground="#111")
        self.driverFrame.grid(row=3, column=0, padx=10, pady=2, sticky=N + S)
        self.l3= Label(self.servicFrame, text="Upload the google API service credentials:")
        self.l3.grid(row=2, column=0)
        self.l1 = Label(self.driverFrame, text="Upload Chromedriver:")
        self.l1.grid(row=3, column=0)
        self.l4=Label(self.emailFrame, text="Enter your mail id:")
        self.l4.grid(row=1, column=0)
        self.l5= Label(self.frame, text="No File Selected",bg = "white", pady = 5)
        self.l5.grid(row=2, column=1)
        self.l7 = Label(self.frame, text="No File Selected", bg="white", pady=5)
        self.l7.grid(row=3, column=1)
        self.E1 = Entry(self.frame, bd=5)
        self.E1.grid(row=1, column=1)
        self.btn2= Button(self.frame, text="Submit", command=self.sci_thread)
        self.btn2.grid(row=5, column=0)
        self.btn3 = Button(self.frame, text="Close", command=self.frame.destroy)
        self.btn3.grid(row=5, column=1)
        self.btn1 = Button(self.frame, text='Browse Credentials', command=self.browse_button)
        self.btn1.grid(row=2, column=2)
        self.btn4 = Button(self.frame, text='Browse Driver', command=self.browse_button_1)
        self.btn4.grid(row=3, column=2)
        self.l6 = Label(self.frame, text="Click Submit to Update",bg = "white")
        self.l6.grid(row=6,column=0)
        self.frame.mainloop()
    def proces(self):
        # Verify that the data entered is correct or not before submitting
        try:
            global email
            email=str(Entry.get(self.E1))
            text=self.l5.cget("text")
            text2=self.l7.cget("text")
            # When no text is entered in any entry and no file is selected raise this warning
            if text2=="No File Selected" or email==""or text=="No File Selected" or("@" not in email):
                if email=="" or "@" not in email:
                    tkMessageBox.showwarning("Warning", "Please enter a valid Ticker, API Key and Email id")
                if "No File" in text:
                    tkMessageBox.showwarning("Warning", "Please select google credentials file")
                if "No File" in text2:
                    tkMessageBox.showwarning("Warning", "Please select the Chromedriver")
            else:
                self.scraping()
                self.l6.config(text="Data Updated")
                # This calls the google sheets function
        except:
            tkMessageBox.showwarning("Warning", "Please enter Valid data")

    def sci_thread(self):
        maincal = threading.Thread(target=self.proces)
        maincal.start()
    def browse_button(self):
        # This function act as submit button for google credentials
        filename1 = filedialog.askopenfilename(initialdir="/", title="Select file",filetypes=(("json files", "*.json"), ("all files", "*.*")))
        if filename1:
            self.l5.config(text=filename1)
    def browse_button_1(self):
        # This function act as submit button for chromedriver
        filename1 = filedialog.askopenfilename(initialdir="/", title="Select file",filetypes=(("exe files", "*.exe"), ("all files", "*.*")))
        if filename1:
            self.l7.config(text=filename1)

    def scraping(self):
        chrome_options =webdriver.ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications": 2}
        chrome_options.add_experimental_option("prefs", prefs)
        driver = Chrome(executable_path=self.l7.cget("text"),options=chrome_options)
        # The line of code below calls the webdriver and opens this website
        driver.get('https://www.hdfcfund.com/statutory-disclosure/aum')
        time.sleep(3)
        wait = WebDriverWait(driver, 15)
        click_element = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='app']/div/div[3]/div[1]/div/div/div[2]/button[1]")))
        click_element.click()
        #It clicks the intial box for foreign or Indian National box
        dropdown = wait.until(EC.presence_of_element_located(
            (By.XPATH, "/html/body/div[1]/div/div[2]/div/div/div[3]/div[1]/div/div/div")))
        dropdown.click()
        monthly = wait.until(EC.presence_of_element_located(
            (By.XPATH, "/html/body/div[1]/div/div[2]/div/div/div[3]/div[1]/div/div/div/ul/div/li[1]")))
        monthly.click()
        # This clicks on the monthly option for AUM Data
        page_view = wait.until(EC.presence_of_all_elements_located(
            (By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div[3]/div[3]/div/span')))
        time.sleep(2)
        #driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        page_view = len(page_view)
        view = 1
        month = []
        years = []
        all_links = []
        while view <= page_view:
            #This while loop is for collecing all the excel sheets on the page then change the page at the bottom
            time.sleep(1)
            number = wait.until(EC.presence_of_element_located(
                (By.XPATH, f'/html/body/div[1]/div/div[2]/div/div/div[3]/div[3]/div/span[{view}]')))
            if number.text == ">":
                actions = ActionChains(driver)
                actions.click(number).perform()
                time.sleep(4)
                page_view = wait.until(EC.presence_of_all_elements_located(
                    (By.XPATH, f'/html/body/div[1]/div/div[2]/div/div/div[3]/div[3]/div/span')))
                page_view = len(page_view)
                view = 0
                view = view + 2
                time.sleep(1)
            else:
                actions = ActionChains(driver)
                actions.click(number).perform()
                time.sleep(5)
                page_data = wait.until(EC.presence_of_all_elements_located(
                    (By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div[3]/div[2]/div/div')))
                page_data = len(page_data)
                # print(page_data)
                #print("reached in else")
                time.sleep(2)
                #k = list(map(link_collect,self.month,self.years,self.all_links, range(1, page_data + 1)))
                for dat in range(1,page_data+1):
                    data = wait.until(EC.presence_of_element_located(
                        (By.XPATH, f'/html/body/div[1]/div/div[2]/div/div/div[3]/div[2]/div/div[{dat}]/a')))
                    year = data.text
                    #print("reached in link function")
                    month.append(year)
                    #print("month accessed")
                    year = re.sub("[^0-9]", "", year)
                    link = data.get_attribute("href")
                    all_links.append(link)
                    years.append(year)

                #print("after k")
                view = view + 1
                time.sleep(2)
                # time.sleep(2)
        driver.quit()
        print(month)
        print(years)
        all_links = create_dict(all_links,years)
        month = create_dict(month,years)
        gc = pygsheets.authorize(service_file=self.l5.cget("text"))
        for year in all_links:
            # This function create google sheets
            res = gc.sheet.create(f'HDFC AMC Net AUM Scheme-wise for {year}')
            res1 = gc.sheet.create(f'HDFC AMC Statewise Contribution to AUM for {year}')
            aum_sheet = gc.open_by_key(res['spreadsheetId'])
            aum_sheet.share(str(Entry.get(self.E1)), role='writer', type='user')
            city_sheet = gc.open_by_key(res1['spreadsheetId'])
            city_sheet.share(str(Entry.get(self.E1)), role='writer', type='user')
            for index in range(len(all_links[year])):
                file_generate(all_links[year][index], month[year][index], index, aum_sheet, city_sheet)



def main():
    gui = GUI_python()
    tar = gui.gui_guide()
if __name__ == "__main__":
    main()
